In [46]:
import pandas as pd
import codecs
truth_file=open('ground_truth_pairs.tsv', 'r')
truth=pd.read_csv(truth_file,header=None,names=['queryid','docid','target'],sep='\t')
truth_file.close()
prediction_file=codecs.open('predictions_2_7.tsv', 'r',encoding='utf-8')
prediction=pd.read_csv(prediction_file,header=None,names=['queryid','docid','sim','details','query','category','title'],sep='\t')
prediction_file.close()
match_data=pd.merge(truth,prediction,on=['queryid','docid'],how='left')
print(match_data.head(5))

   queryid    docid  target        sim  \
0      119  1000015       1  10.733849   
1      134  1000055      -1  26.299755   
2       24  1000061      -1        NaN   
3      111  1000075       1  11.925878   
4      142  1000082      -1        NaN   

                                             details          query  \
0  {'id': '1000015', 'title': 'kotobukiya art fx ...  batman statue   
1  {'id': '1000055', 'title': 'us airways pilot t...   pilot helmet   
2                                                NaN            NaN   
3  {'id': '1000075', 'title': 'tigger bank walt d...    disney bank   
4                                                NaN            NaN   

                                            category  \
0                  Collectibles > Comics > Figurines   
1   Collectibles > Transportation > Aviation > Ai...   
2                                                NaN   
3   Collectibles > Disneyana > Contemporary (1968...   
4                                       

In [51]:
match_data['prediction']=pd.notna(match_data['sim']).map({True: 1, False: -1})

In [52]:
print(match_data.head(5))

   queryid    docid  target        sim  \
0      119  1000015       1  10.733849   
1      134  1000055      -1  26.299755   
2       24  1000061      -1        NaN   
3      111  1000075       1  11.925878   
4      142  1000082      -1        NaN   

                                             details          query  \
0  {'id': '1000015', 'title': 'kotobukiya art fx ...  batman statue   
1  {'id': '1000055', 'title': 'us airways pilot t...   pilot helmet   
2                                                NaN            NaN   
3  {'id': '1000075', 'title': 'tigger bank walt d...    disney bank   
4                                                NaN            NaN   

                                            category  \
0                  Collectibles > Comics > Figurines   
1   Collectibles > Transportation > Aviation > Ai...   
2                                                NaN   
3   Collectibles > Disneyana > Contemporary (1968...   
4                                       

In [40]:
query_cats={}
for queryid in range(1,151):
    query_cats[queryid]=dict()
query_cats_sum={}
for queryid in range(1,151):
    query_cats_sum[queryid]=dict()
counter=0
for index,row in match_data.iterrows():
    queryid=row['queryid']
    cat=[x.strip() for x in row['category'].split(">")]
    target=row['target']
    #print(queryid, cat,target)
    for j in range(2,len(cat)+1):
        subcat=(" > ".join(cat[0:j])).lower()
        if subcat in query_cats[queryid]:
            query_cats[queryid][subcat]+=1
            if target==1:
                query_cats_sum[queryid][subcat]+=1
        else:
            query_cats[queryid][subcat]=1
            if target==1:
                query_cats_sum[queryid][subcat]=1
            else:
                query_cats_sum[queryid][subcat]=0
    counter=counter+1
    #if counter>100000:
    #    break


query_cats_exclude={}
for queryid in range(1,151):
    query_cats_exclude[queryid]=set()
for queryid in query_cats:
    for subcat in query_cats[queryid]:
        total=query_cats[queryid][subcat]
        if total<5:
            continue
        relevant=query_cats_sum[queryid][subcat]
        if relevant/total<0.5:
            query_cats_exclude[queryid].add(subcat)

In [45]:
for queryid in range(1,21):
    print(query_cats_exclude[queryid])

{'collectibles > non-sport trading cards'}
{'collectibles > comics > bronze age (1970-83) > war', 'collectibles > comics > copper age (1984-1991) > war'}
set()
set()
{'collectibles > cultures & ethnicities > native american: us > 1935-now', 'collectibles > cultures & ethnicities > native american: us > 1935-now > sculptures', 'collectibles > cultures & ethnicities > native american: us'}
{'collectibles > autographs', 'collectibles > militaria > ww ii (1939-45) > reproductions', 'collectibles > photographic images > contemporary (1940-now)', 'collectibles > advertising', 'collectibles > militaria > current militaria (2001-now) > original items > personal, field gear > pouches', 'collectibles > militaria > ww ii (1939-45) > reproductions > united states'}
set()
set()
set()
{'collectibles > disneyana > contemporary (1968-now) > pins, patches & buttons > disney characters & movies'}
set()
set()
{'collectibles > postcards', 'collectibles > cultures & ethnicities > latin american > mexico > 

In [44]:
prediction_file=codecs.open('predictions_2_7.tsv', 'r',encoding='utf-8')
prediction=pd.read_csv(prediction_file,header=None,names=['queryid','docid','sim','details','query','category','title'],sep='\t')
prediction_file.close()
out_file=codecs.open('predictions_cats_exclude.tsv', 'w',encoding='utf-8')
for index,row in prediction.iterrows():
    queryid=row['queryid']
    #if queryid>10:
    #    break
    docid=row['docid']
    cat=[x.strip() for x in row['category'].split(">")]
    to_exclude=0
    for j in range(2,len(cat)+1):
        subcat=(" > ".join(cat[0:j])).lower()
        if subcat in query_cats_exclude[queryid]:
            to_exclude=1
    if to_exclude:
        out_file.write(str(queryid)+"\t"+str(docid)+"\t"+"0"+"\t"+str(cat)+"\n")
    else:
        out_file.write(str(queryid)+"\t"+str(docid)+"\t"+"1\n")
out_file.close()  

In [84]:
new_prediction_file=codecs.open('predictions_cats_exclude.tsv', 'r',encoding='utf-8')
new_prediction=pd.read_csv(new_prediction_file,header=None,names=['queryid','docid','prediction','details'],sep='\t')
new_prediction_file.close()
new_prediction=new_prediction[new_prediction['prediction']==1.0]
print(new_prediction.shape)
new_match_data=pd.merge(truth,new_prediction,on=['queryid','docid'],how='left')
new_match_data['prediction']=pd.notna(new_match_data['prediction']).map({True: 1, False: -1})
print(new_match_data.head(5))

(294961, 4)
   queryid    docid  target  prediction details
0      119  1000015       1           1     NaN
1      134  1000055      -1           1     NaN
2       24  1000061      -1          -1     NaN
3      111  1000075       1           1     NaN
4      142  1000082      -1          -1     NaN
